In [ ]:
# 1. Importació de llibreries i configuració inicial

from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Obrim les llibreries necessàries

import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, ArrayType


In [ ]:
# Obrim Spark Session

spark = SparkSession.builder.appName("HW2_Recomanador_Amics").getOrCreate()


In [ ]:
# 2. Configuració de directoris (muntar Google Drive si és necessari)

# Definim el camí del fitxer
file_path = "/content/drive/MyDrive/HW2/soc-LiveJournal1Adj.txt"


In [ ]:
# 3. Càrrega i lectura del fitxer d'amistats
# Carreguem el fitxer com a RDD
data_rdd = spark.sparkContext.textFile(file_path)
# Mostrem algunes línies per comprovar
data_rdd.take(5)


['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878',
 '3\t0,12,41,55,1532,12636,13185,27552,38737',
 '4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822']

In [ ]:
# 4. Preprocessament: Transformació de dades

# Separem cada línia en (usuari, llista d'amics)
user_friends_rdd = data_rdd.map(lambda line: line.split("\t")) \
    .filter(lambda x: len(x) == 2 and x[1].strip() != "") \
    .map(lambda x: (int(x[0]), list(map(int, x[1].split(",")))))
user_friends_rdd.take(5)


[(0,
  [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94]),
 (1,
  [0,
   5,
   20,
   135,
   2409,
   8715,
   8932,
   10623,
   12347,
   12846,
   13840,
   13845,
   14005,
   20075,
   21556,
   22939,
   23520,
   28193,
   29724,
   29791,
   29826,
   30691,
   31232,
   31435,
   32317,
   32489,
   34394,
   35589,
   35605,
   35606,
   35613,
   35633,
   35648,
   35678,
   38737,
   43

In [ ]:
# 5. Generació de parelles d'amics i amics d'amics

# Per cada usuari, generem parelles (usuari, amic)
pairs_rdd = user_friends_rdd.flatMap(lambda x: [(x[0], friend) for friend in x[1]])
pairs_rdd.take(5)


[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]

In [ ]:
# Generem possibles amics futurs: (usuari, amic d'amic)

potential_friends_rdd = user_friends_rdd.flatMap(
    lambda x: [(friend, other_friend) for friend in x[1] for other_friend in x[1] if friend != other_friend]
)
potential_friends_rdd.take(5)


[(1, 2), (1, 3), (1, 4), (1, 5), (1, 6)]

In [ ]:
# 6. Eliminació de parelles d'amistats existents
# Marquem els amics existents amb valor -1
existing_friendships_rdd = pairs_rdd.map(lambda x: ((x[0], x[1]), -1))
existing_friendships_rdd.take(5)


[((0, 1), -1), ((0, 2), -1), ((0, 3), -1), ((0, 4), -1), ((0, 5), -1)]

In [ ]:
# Marquem les recomanacions potencials amb valor 1

potential_recommendations_rdd = potential_friends_rdd.map(lambda x: ((x[0], x[1]), 1))
potential_recommendations_rdd.take(5)


[((1, 2), 1), ((1, 3), 1), ((1, 4), 1), ((1, 5), 1), ((1, 6), 1)]

In [ ]:

# Combinar amistats existents i potencials
all_relations_rdd = existing_friendships_rdd.union(potential_recommendations_rdd)
relations_count_rdd = all_relations_rdd.reduceByKey(lambda x, y: x + y)
all_relations_rdd.take(10)


[((0, 1), -1),
 ((0, 2), -1),
 ((0, 3), -1),
 ((0, 4), -1),
 ((0, 5), -1),
 ((0, 6), -1),
 ((0, 7), -1),
 ((0, 8), -1),
 ((0, 9), -1),
 ((0, 10), -1)]

In [ ]:
# Filtramos solo recomendaciones

valid_recommendations_rdd = relations_count_rdd.filter(lambda x: x[1] > 0)


[((1, 38737), 3),
 ((35605, 35621), 26),
 ((35613, 35621), 21),
 ((38737, 5933), 1),
 ((29581, 35477), 1),
 ((2406, 2412), 2),
 ((2678, 2668), 7),
 ((10240, 19650), 3),
 ((83, 34211), 1),
 ((2246, 32), 1)]

In [ ]:
# 7. Organitzar recomanacions per usuari

# Convertim a (usuari, (amic_recomanat, nombre d'amics en comú))
recommendations_rdd = valid_recommendations_rdd.map(
    lambda x: (x[0][0], (x[0][1], x[1]))
)

grouped_recommendations_rdd = recommendations_rdd.groupByKey()


In [ ]:
# Mostrar recomendaciones de los primeros 10 usuarios
for user, recs in grouped_recommendations_rdd.take(10):
    print(f"Usuari {user}:\n")
    for friend, count in recs:
        print(f"  Recomanat: {friend} (amics en comú: {count})")
    print()


Usuari 23520:

  Recomanat: 35570 (amics en comú: 4)
  Recomanat: 25186 (amics en comú: 2)
  Recomanat: 42622 (amics en comú: 1)
  Recomanat: 43158 (amics en comú: 1)
  Recomanat: 10318 (amics en comú: 1)
  Recomanat: 25138 (amics en comú: 3)
  Recomanat: 42062 (amics en comú: 1)
  Recomanat: 25202 (amics en comú: 1)
  Recomanat: 39242 (amics en comú: 1)
  Recomanat: 16466 (amics en comú: 1)
  Recomanat: 27538 (amics en comú: 1)
  Recomanat: 27750 (amics en comú: 1)
  Recomanat: 14218 (amics en comú: 1)
  Recomanat: 44366 (amics en comú: 1)
  Recomanat: 5222 (amics en comú: 1)
  Recomanat: 14058 (amics en comú: 3)
  Recomanat: 34426 (amics en comú: 2)
  Recomanat: 45150 (amics en comú: 1)
  Recomanat: 13938 (amics en comú: 6)
  Recomanat: 32534 (amics en comú: 2)
  Recomanat: 14002 (amics en comú: 1)
  Recomanat: 35646 (amics en comú: 1)
  Recomanat: 41886 (amics en comú: 1)
  Recomanat: 3734 (amics en comú: 2)
  Recomanat: 34538 (amics en comú: 1)
  Recomanat: 13998 (amics en comú: 1)

In [ ]:
# 8. Selecció de les millors recomanacions

# Per cada usuari, seleccionem màxim 10 recomanacions ordenades
top_10_recommendations_rdd = grouped_recommendations_rdd.mapValues(
    lambda recs: [
        friend for friend, common_friends in sorted(
            recs,
            key=lambda x: (-x[1], x[0])
        )[:10]
    ]
)


# 9. Format final del resultat
# Preparem el format final: <UserID> <RecommendedUser1,RecommendedUser2,...>
final_result_rdd = top_10_recommendations_rdd.map(
    lambda x: f"{x[0]}\t{','.join(map(str, x[1]))}"
)
final_result_rdd.take(10)


['23520\t17636,13948,35621,43268,13960,23510,23512,35605,35630,49226',
 '9892\t14324,1201,12805,13306,8468,22312,22528,43061,576,3931',
 '27808\t1100,18163,439,27736,13818,1085,19077,1135,4133,25186',
 '9436\t42967,18576,4981,18601,9439,9446,9447,12598,7024,9441',
 '60\t0,56,57,58,59,61,1,2,3,4',
 '36816\t36680,36683,36765,37156,10072,36905,10096,36988,10097,10114',
 '36848\t37215,41851,30220,36735,10066,10018,36672,36703,37150,37580',
 '37228\t36687,10058,36698,37199,37242,37597,36763,36909,37018,37207',
 '37448\t44178,18912,44049,10018,19365,10022,30220,10028,35179,41903',
 '168\t1694,2687,3231,1670,1688,1657,2647,2711,14746,29558']

In [ ]:
target_users = {924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993}

filtered_users_rdd = grouped_recommendations_rdd.filter(lambda x: x[0] in target_users)

for user, recs in filtered_users_rdd.collect():
    print(f"Usuari {user}:\n")
    for friend, count in recs:
        print(f"  Recomanat: {friend} (amics en comú: {count})")
    print()

Usuari 9992:

  Recomanat: 9990 (amics en comú: 2)
  Recomanat: 9994 (amics en comú: 2)
  Recomanat: 9993 (amics en comú: 2)
  Recomanat: 9989 (amics en comú: 4)
  Recomanat: 9988 (amics en comú: 2)
  Recomanat: 9987 (amics en comú: 4)
  Recomanat: 9991 (amics en comú: 2)
  Recomanat: 35667 (amics en comú: 3)

Usuari 9020:

  Recomanat: 9022 (amics en comú: 2)
  Recomanat: 9018 (amics en comú: 1)
  Recomanat: 317 (amics en comú: 1)
  Recomanat: 9017 (amics en comú: 2)
  Recomanat: 9021 (amics en comú: 3)
  Recomanat: 9016 (amics en comú: 2)
  Recomanat: 320 (amics en comú: 1)
  Recomanat: 9023 (amics en comú: 1)
  Recomanat: 9019 (amics en comú: 1)

Usuari 924:

  Recomanat: 2409 (amics en comú: 1)
  Recomanat: 45881 (amics en comú: 1)
  Recomanat: 11860 (amics en comú: 1)
  Recomanat: 43748 (amics en comú: 1)
  Recomanat: 15416 (amics en comú: 1)
  Recomanat: 439 (amics en comú: 1)
  Recomanat: 6995 (amics en comú: 1)

Usuari 9021:

  Recomanat: 317 (amics en comú: 1)
  Recomanat: 901

In [ ]:
# 10. (Opcional) Guardar els resultats a Google Drive

# Definim el camí de sortida
output_path = "/content/drive/MyDrive/HW2/output_recommendations"

# Si ja existeix una carpeta amb aquest nom, Spark donarà error.
# Per tant, si cal, primer esborrem la carpeta anterior (només per a proves, amb precaució!)
import shutil
import os
import glob

if os.path.exists(output_path):
    shutil.rmtree(output_path)

# Guardem els resultats com a arxius de text
final_result_rdd.saveAsTextFile(output_path)

# Buscar el archivo de datos real (el part-xxxxx)
part_file = glob.glob(output_path + "/part-*")[0]

# Definir ruta de destino final con extensión .txt
final_txt = "/content/drive/MyDrive/HW2/resultados.txt"

# Copiar el archivo como .txt
shutil.copy(part_file, final_txt)

print("Guardado como resultados.txt correctamente.")


Guardado como resultados.txt correctamente.
